In [3]:
#排队模拟

import simpy
import random
import statistics

#声明从进入电影院到检票完成需要的时间变量
wait_times = []

#建立系统模型，模型可以是许多场景，如银行、检查站、洗车店等，此处为电影院
class Theater(object):
    def __init__(self, env, num_cashiers, num_servers, num_ushers):
        self.env = env
        self.cashier = simpy.Resource(env, num_cashiers)
        self.server = simpy.Resource(env, num_servers)
        self.usher = simpy.Resource(env, num_ushers)
        
    def purchase_ticket(self, moviegoer):
        #根据历史数据，一般购取票平均花费1-2分钟
        yield self.env.timeout(random.randint(1,3))
        
    def check_ticket(self, moviegoer):
        #检票平均需要3秒钟，转为分钟数
        yield self.env.timeout(3/60)
    
    def sell_food(self, moviegoer):
        #购买饮食，平均花费1-5分钟
        yield self.env.timeout(random.randint(1,5))

def go_to_movies(env, moviegoer, theater):
    #观众达到影院的时间
    arrival_time = env.now
    
    #等待购取票 
    with theater.cashier.request() as request:  #观众向收银员请求购票
        yield request    #观众等待收银员完成前面的服务
        yield env.process(theater.purchase_ticket(moviegoer)) 
        
    #请求检票
    with theater.usher.request() as request:
        yield request
        yield env.process(theater.check_ticket(moviegoer))
        
    #不是每一个观众都会购买饮食，这里随机选择是否购买
    if random.choice([True, False]):
        with theater.server.request() as request:
            yield request
            yield env.process(theater.sell_food(moviegoer))
            
    #观众进入会场，前后共计时间
    wait_times.append(env.now - arrival_time)

def run_theater(env, num_cashiers, num_servers, num_ushers):
    theater = Theater(env, num_cashiers, num_servers, num_ushers)
    
    for moviegoer in range(3):#初始化设置，开始有3个观众在等待购取票
        env.process(go_to_movies(env, moviegoer, theater))
        
    while True:
        yield env.timeout(0.20)#此处假设每0.2分钟新增一个观众进入电影院
        
        moviegoer += 1
        env.process(go_to_movies(env, moviegoer, theater))

#定义函数，统计观众平均花费时间，从到达影院到完成检票
def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)
    #返回等待时间的分秒数，方便查看
    minutes, frac_minutes = divmod(average_wait, 1)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds) 

#创建一个函数，用户可以自主选择员工数量
def get_user_input():
    num_cashiers = input("Input # of cashiers working:")
    num_servers = input("Input # of servers working:")
    num_ushers = input("Input # of ushers working")
    params = [num_cashiers, num_servers, num_ushers]
    if all (str(i).isdigit() for i in params):#Cheack iinput is valid
        params = [int(x) for x in params]
    else:
        print("Could not parse input.The simulation will use default values;",
             "\n1 cashier, 1 server, 1 usher.",
             )
        params = [1,1,1]
    return params

def main():
    #初始化
    random.seed(42)
    num_cashiers, num_servers, num_ushers = get_user_input()
    
    #运行模拟场景
    env = simpy.Environment()
    env.process(run_theater(env, num_cashiers, num_servers, num_ushers))
    env.run(until=90)#假设观众持续入场90分钟
    
    #查看统计结果
    mins, secs = get_average_wait_time(wait_times)
    print(
    "运行程序...",
    f"\n平均等待时间是{mins} 分钟和 {secs} 秒。")

if __name__ == '__main__':
    main()


Input # of cashiers working:100
Input # of servers working:100
Input # of ushers working100
运行程序... 
平均等待时间是3 分钟和 29 秒。


In [4]:
"""
服务站示例
场景介绍:
  一个有特定服务提供工作站，客户服务时长不一，工作机器数有限。
  Client接受服务步骤：Client到达工作站，若有空闲的机器就立刻接受服务，如果没有，就等待直到其他机器空闲下来。
  每个接受过服务的Client都有一个完成满意度（或者为进度）实时统计服务客户数和完成满意进度。
"""
import random
import simpy
 
# 可接受输入参数
RANDOM_SEED = 0         # 不设置
NUM_MACHINES = 2        # 可以同时处理的机器数（类似工作工位数）
TIME_CONSUMING = 5      # 单任务耗时 (可以设计成随机数)
TIME_INTERVAL = 5       # 来车的间隔时间约5分钟   (可以设计成随机数)
SIM_TIME = 1000         # 仿真总时间
CLIENT_NUMBER = 2       # 初始时已经占用机器数
 
 
class WorkStation(object):
    """
    一个工作站，拥有特定数量的机器数。 一个客户首先申请服务。在对应服务时间完成后结束并离开工作站
    """
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime
        self.allClient = 0
        self.accomplishClient = 0
 
    def wash(self, car):
        """服务流程"""
        yield self.env.timeout(random.randint(2, 10))  # 假设服务时间为随机数（2~10）
        self.allClient += 1
        per = random.randint(50, 99)
        print("%s's 任务完成度：%d%%." % (car, per))
        if per > 80:
            self.accomplishClient += 1
 
        print("工作站服务客户数：%d,"
              "工作站服务达标率：%.2f。" % (self.allClient, float(self.accomplishClient) / float(self.allClient)))
 
 
def Client(env, name, cw):
    """
    客户到达动作站接受服务，结束后离开
    """
 
    print('%s 到达工作站 at %.2f.' % (name, env.now))
    with cw.machine.request() as request:
        yield request
        print('%s 接受服务   at %.2f.' % (name, env.now))
        yield env.process(cw.wash(name))
        print('%s 离开服务站 at %.2f.' % (name, env.now))
 
 
def setup(env, num_machines, washtime, t_inter, clientNumber):
    """创建一个工作站，几个初始客户，然后持续有客户到达. 每隔t_inter - 2, t_inter + 3分钟（可以自定义）."""
    # 创建工作站
    workstation = WorkStation(env, num_machines, washtime)
 
    # 创建clientNumber个初始客户
    for i in range(clientNumber):
        env.process(Client(env, 'Client_%d' % i, workstation))
 
    # 在仿真过程中持续创建客户
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 3))  # 3-8分钟
        i += 1
        env.process(Client(env, 'Client_%d' % i, workstation))
 
 
# 初始化并开始仿真任务
print('开始仿真')
 
# 初始化seed，指定数值的时候方正结果可以复现
random.seed()
 
# 创建一个环境并开始仿真
env = simpy.Environment()
env.process(setup(env, NUM_MACHINES, TIME_CONSUMING, TIME_INTERVAL, CLIENT_NUMBER))
 
# 开始执行!
env.run(until=SIM_TIME)

开始仿真
Client_0 到达工作站 at 0.00.
Client_1 到达工作站 at 0.00.
Client_0 接受服务   at 0.00.
Client_1 接受服务   at 0.00.
Client_2 到达工作站 at 7.00.
Client_0's 任务完成度：62%.
工作站服务客户数：1,工作站服务达标率：0.00。
Client_1's 任务完成度：90%.
工作站服务客户数：2,工作站服务达标率：0.50。
Client_0 离开服务站 at 10.00.
Client_1 离开服务站 at 10.00.
Client_2 接受服务   at 10.00.
Client_3 到达工作站 at 11.00.
Client_3 接受服务   at 11.00.
Client_3's 任务完成度：73%.
工作站服务客户数：3,工作站服务达标率：0.33。
Client_3 离开服务站 at 13.00.
Client_2's 任务完成度：99%.
工作站服务客户数：4,工作站服务达标率：0.50。
Client_4 到达工作站 at 17.00.
Client_2 离开服务站 at 17.00.
Client_4 接受服务   at 17.00.
Client_5 到达工作站 at 24.00.
Client_5 接受服务   at 24.00.
Client_4's 任务完成度：66%.
工作站服务客户数：5,工作站服务达标率：0.40。
Client_4 离开服务站 at 26.00.
Client_6 到达工作站 at 29.00.
Client_6 接受服务   at 29.00.
Client_6's 任务完成度：81%.
工作站服务客户数：6,工作站服务达标率：0.50。
Client_6 离开服务站 at 31.00.
Client_5's 任务完成度：71%.
工作站服务客户数：7,工作站服务达标率：0.43。
Client_7 到达工作站 at 34.00.
Client_5 离开服务站 at 34.00.
Client_7 接受服务   at 34.00.
Client_7's 任务完成度：65%.
工作站服务客户数：8,工作站服务达标率：0.38。
Client_7 离开服务站 at 39.00.
Client_8

工作站服务客户数：187,工作站服务达标率：0.35。
Client_186 离开服务站 at 978.00.
Client_187's 任务完成度：62%.
工作站服务客户数：188,工作站服务达标率：0.35。
Client_187 离开服务站 at 982.00.
Client_188 到达工作站 at 983.00.
Client_188 接受服务   at 983.00.
Client_188's 任务完成度：76%.
工作站服务客户数：189,工作站服务达标率：0.35。
Client_188 离开服务站 at 988.00.
Client_189 到达工作站 at 990.00.
Client_189 接受服务   at 990.00.
Client_190 到达工作站 at 993.00.
Client_190 接受服务   at 993.00.
Client_189's 任务完成度：93%.
工作站服务客户数：190,工作站服务达标率：0.35。
Client_191 到达工作站 at 996.00.
Client_189 离开服务站 at 996.00.
Client_191 接受服务   at 996.00.
Client_190's 任务完成度：72%.
工作站服务客户数：191,工作站服务达标率：0.35。
Client_190 离开服务站 at 998.00.
Client_191's 任务完成度：67%.
工作站服务客户数：192,工作站服务达标率：0.35。
Client_191 离开服务站 at 999.00.


In [1]:
#example1

import simpy
def car(env):#定义汽车进程生成器 传入环境实例env
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)#被挂起并等待超时执行
 
        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)
        
env = simpy.Environment()#声明环境实例
env.process(car(env))#传入进程
env.run(until=15)#进程开始

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


In [2]:
#引入资源概念

import simpy
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)#实例化两个资源

def car(env, name, bcs, driving_time, charge_duration):
    # 模拟驶向充电站
    yield env.timeout(driving_time)
 
    # 请求充点桩
    print('%s 到达时间： %d' % (name, env.now))
    with bcs.request() as req:
        yield req
 
        # 充点进程
        print('%s 充点开始时间： %s' % (name, env.now))
        yield env.timeout(charge_duration)
        print('%s 驶离时间： %s' % (name, env.now))

#创建汽车进程        
for i in range(4):
    env.process(car(env, '第 %d 辆汽车 ' % (i+1), bcs, i*2, 5))

env.run()


第 1 辆汽车  到达时间： 0
第 1 辆汽车  充点开始时间： 0
第 2 辆汽车  到达时间： 2
第 2 辆汽车  充点开始时间： 2
第 3 辆汽车  到达时间： 4
第 1 辆汽车  驶离时间： 5
第 3 辆汽车  充点开始时间： 5
第 4 辆汽车  到达时间： 6
第 2 辆汽车  驶离时间： 7
第 4 辆汽车  充点开始时间： 7
第 3 辆汽车  驶离时间： 10
第 4 辆汽车  驶离时间： 12
